# Transformer - Probabilistic Forecast

## 0. imports

In [1]:
%load_ext jupyter_black

In [2]:
import sys

sys.path.append("..")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import namedtuple

In [4]:
from src.dataset import ETTDataModule
from src.model import Transformer

## 1. prev setting

In [5]:
dm_params = {
    "data_path": "../data/ETT-small/ETTh1.csv",
    "task": "M",
    "freq": "h",
    "target": "OT",
    "seq_len": 96,
    "label_len": 48,
    "pred_len": 96,
    "use_scaler": True,
    "use_time_enc": True,
    "batch_size": 32,
}


dm = ETTDataModule(**dm_params)

In [6]:
# transformer_params
Config = namedtuple(
    "Config",
    [
        "task_name",
        "pred_len",
        "seq_len",
        "num_class",
        "enc_in",
        "dec_in",
        "c_out",
        "d_model",
        "embed_type",
        "freq",
        "dropout",
        "n_heads",
        "d_keys",
        "d_values",
        "d_ff",
        "scale",
        "attention_dropout",
        "output_attention",
        "activation",
        "num_enc_layers",
        "num_dec_layers",
    ],
)

configs = Config(
    task_name="long_term_forecast",
    pred_len=96,
    seq_len=None,
    num_class=None,
    enc_in=7,
    dec_in=7,
    c_out=7,
    d_model=512,
    embed_type="time_features",
    freq="h",
    dropout=0.1,
    n_heads=8,
    d_keys=None,
    d_values=None,
    d_ff=2048,
    scale=None,
    attention_dropout=0.1,
    output_attention=True,
    activation="gelu",
    num_enc_layers=2,
    num_dec_layers=1,
)

In [7]:
model = Transformer(**configs._asdict())

In [8]:
train_dataloader = dm.train_dataloader()
batch = next(iter(train_dataloader))

# decoder input
label_len = 48
dec_inp = torch.zeros_like(batch["future_values"][:, -configs.pred_len :, :]).float()
dec_inp = torch.cat([batch["future_values"][:, :label_len, :], dec_inp], dim=1).float()

In [9]:
output = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    future_values=dec_inp,
    future_time_features=batch["future_time_features"],
)

In [13]:
output["last_hidden_states"]

torch.Size([32, 144, 7])

In [15]:
batch["future_values"].shape

torch.Size([32, 144, 7])